### Imports

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
#import matplotlib.pyplot as plt
#from feature_baggingV2 import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import datetime
from sklearn.preprocessing import RobustScaler
#from keras.models import Sequential
#from keras.layers import LSTM, Dense
import labolibrary as labo

import os, sys, gc, time, warnings, pickle, psutil, random

import warnings
warnings.filterwarnings("ignore")

### Función para escalar/desescalar y métrica

In [ ]:
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler(with_centering=False)
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)

### Archivos

In [ ]:
#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'
df_sell_in = pd.read_csv(DATOS_DIR+'sell-in.txt', sep='\t')
df_predecir = pd.read_csv(DATOS_DIR+'productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv(DATOS_DIR+'tb_stocks.txt', sep='\t')
df_tb_productos = pd.read_csv(DATOS_DIR+'tb_productos_descripcion.txt', sep='\t')

### Preprocesamiento

In [ ]:
#AGREGAR VENTAS EN CERO

df_group = pl.DataFrame(df_sell_in)
df_group = df_group.group_by(['periodo','product_id','customer_id']).agg(pl.sum('tn').alias('tn_sum'))

# Fechas de inicio y fin de cada producto
df_fechas = df_group.group_by('product_id').agg([
                                            pl.min('periodo').alias('periodo_min'),
                                            pl.max('periodo').alias('periodo_max')])


a = df_group.group_by(['product_id','customer_id']).agg(pl.count('periodo').alias('periodo_count'))
a = a.filter(pl.col('periodo_count') >= 6)
df_group = df_group.join(a, on=['product_id', 'customer_id'], how='inner')



In [ ]:



df_group.group_by(['product_id','customer_id']).agg(pl.count('periodo').alias('periodo_count')).sort('periodo_count', descending=True)

In [ ]:
# Obtener valores únicos de customer_id, product_id y periodo
unique_customer_ids = df_sell_in['customer_id'].unique()
unique_product_ids = df_sell_in['product_id'].unique()
unique_periodos = df_sell_in['periodo'].unique()

df_customers = pl.DataFrame({'customer_id': unique_customer_ids})
df_products = pl.DataFrame({'product_id': unique_product_ids})
df_periods = pl.DataFrame({'periodo': unique_periodos})

# Creo las combinaciones con cross join
df_all_combinations = df_customers.join(df_products, how = 'cross').join(df_periods, how= 'cross')

In [ ]:
#Lo uno con los inicios y fin de cada producto
df_all_combinations_2 = df_all_combinations.join(df_fechas, on = 'product_id', how='left')
df_all_combinations_2


In [ ]:
#filtro los que no estan dentro del rango de inicio y fin
filtered_df = df_all_combinations_2.filter((df_all_combinations_2['periodo'] >= df_all_combinations_2['periodo_min']) & (df_all_combinations_2['periodo'] <= df_all_combinations_2['periodo_max']))
filtered_df

In [ ]:
df_sell = pl.DataFrame(df_sell_in)


In [ ]:
# Calcular si cada combinación de product_id y customer_id ha comprado
df_grouped_2 = df_sell.group_by(["product_id", "customer_id"]).agg(pl.count("periodo").alias("count"))

In [ ]:
#Join final con sell in
df_complete = filtered_df.join(df_sell, 
                 left_on=['customer_id', 'product_id','periodo'],
                 right_on=['customer_id', 'product_id','periodo'],
                 how='left'
                )
df_complete = df_complete.drop("plan_precios_cuidados") #Lo elimino porque sino con fill null se coloca en cero
df_complete = df_complete.fill_null(0)



In [ ]:
df_complete = df_complete.join(df_grouped_2,left_on=['customer_id', 'product_id'],
                 right_on=['customer_id', 'product_id'],                 how='left'
                )

In [ ]:
df_complete.filter(pl.col("count") >= 6 ) 

In [ ]:
df_complete = df_complete.filter(pl.col("count") >= 2 ) 
df_complete 

In [ ]:


# Create a copy of the original dataframe to perform operations
df_group_copy = df_complete.clone()

# Ensure 'periodo' is treated as a string for filtering
df_group_copy = df_group_copy.with_columns([
    pl.col('periodo').cast(pl.Utf8)
])

# Define the date range
start_period = "201906"
end_period = "201911"

# Filter to get only rows within the last six months
df_last_six_months = df_group_copy.filter((pl.col('periodo') >= start_period) & (pl.col('periodo') <= end_period))

# Group by product_id and customer_id and calculate the sum of tn
df_summed_tn = df_last_six_months.group_by(['product_id', 'customer_id']).agg(pl.sum('tn').alias('tn_sum'))

# Filter out combinations where the sum of tn is zero
valid_combinations = df_summed_tn.filter(pl.col('tn_sum') > 0).select(['product_id', 'customer_id'])

# Perform an inner join to filter the original dataframe
df_filtered = df_complete.join(valid_combinations, on=['product_id', 'customer_id'], how='inner')

# Display the result
df_complete =df_filtered


Testeo de agregado de ceros

In [ ]:
#Original
df_sell.filter((pl.col("product_id") == 21276) & (pl.col("customer_id")==10550))

In [ ]:
#Modificado
df_complete.filter((pl.col("product_id") == 20001))

In [ ]:
df_complete = df_complete.to_pandas()

In [ ]:
#Defino los formatos
df_tb_stocks['periodo'] = pd.to_datetime(df_tb_stocks['periodo'], format='%Y%m')
df_tb_stocks['periodo'] = df_tb_stocks['periodo'] - pd.DateOffset(months=1) #mes diferente en stock
df_tb_stocks['product_id'] = df_tb_stocks['product_id'].astype(int)
df_tb_stocks['stock_final'] = df_tb_stocks['stock_final'].astype(float)
df_tb_productos['product_id'] = df_tb_productos['product_id'].astype(int)
df_tb_productos['sku_size'] = df_tb_productos['sku_size'].astype(int)
df_complete['periodo'] = pd.to_datetime(df_complete['periodo'], format='%Y%m')
df_complete['product_id'] = df_complete['product_id'].astype(int)
df_complete['customer_id'] = df_complete['customer_id'].astype(int)
df_complete['cust_request_qty'] = df_complete['cust_request_qty'].astype(int)
df_complete['cust_request_tn'] = df_complete['cust_request_tn'].astype(float)
df_complete['tn'] = df_complete['tn'].astype(float)
#df_complete['plan_precios_cuidados'] = df_complete['plan_precios_cuidados'].astype(bool)

In [ ]:
df_sell_in = df_complete

### Consolidar Datos

#### Outliers

In [ ]:
# Define the function to calculate Z-score and cap the outliers
def cap_outliers(df, column_name):
    mean = df[column_name].mean()
    std = df[column_name].std() 
    threshold = 2
    z_scores = (df[column_name] - mean) / std
    df[f'capped_{column_name}'] = np.where(z_scores.abs() > threshold,
                                           mean + threshold * std * np.sign(z_scores),
                                           df[column_name])
    return df

def handle_outliers(df, column_name):
    rolling_median = df[column_name].rolling(window=3, center=True).median()
    diff = np.abs(df[column_name] - rolling_median)
    threshold = 3 * diff.median()
    df[f'cleaned_{column_name}'] = np.where(diff > threshold, rolling_median, df[column_name])
    return df

# Group by product_id and customer_id and apply the function while preserving all columns
df_sell_in = df_sell_in.groupby(['product_id', 'customer_id']).apply(lambda df: cap_outliers(df, 'tn')).reset_index(drop=True)



In [ ]:
df_sell_in[(df_sell_in['product_id'] == 20001) & (df_sell_in['customer_id'] == 10052) & (df_sell_in['periodo'] >= '2019-06-01')][['periodo', 'tn','capped_tn']]

In [ ]:
df_sell_in['tn'] =df_sell_in['capped_tn']

#### Clase a evaluar

In [ ]:
#### Agrupar y escalar
scalers = {}

df_sell_in['weight'] = df_sell_in['tn'] #Variable Weight

df_sell_in['tn'] = df_sell_in.groupby(['product_id','customer_id'])['tn'].transform(scale_group)

#df_sell_in[(df_sell_in['product_id'] == 21276) & (df_sell_in['customer_id'] == 10029)][['periodo', 'tn', 'weight']]
with open(DATOS_DIR+'/scalers.pkl', 'wb') as file:#
     pickle.dump(scalers, file)

In [ ]:
df_sell_in['tn_2'] = df_sell_in.groupby(['product_id', 'customer_id'])['tn'].shift(-2) #METRICA A EVALUAR!
df_sell_in['diff_tn_tn2'] =  df_sell_in['tn_2'] - df_sell_in['tn'] #NUEVA CLASE (NO LA UTILIZAMOS)

In [ ]:
df_sell_in['cust_request_qty'] = df_sell_in.groupby(['product_id','customer_id'])['cust_request_qty'].transform(scale_group) 
df_sell_in['cust_request_tn'] = df_sell_in.groupby(['product_id','customer_id'])['cust_request_tn'].transform(scale_group) 
df_sell_in['cust_request_qty'] = df_sell_in.groupby(['product_id','customer_id'])['cust_request_qty'].transform(scale_group)  
#df_tb_stocks['stock_final'] = df_tb_stocks.groupby(['product_id','customer_id'])['stock_final'].transform(scale_group) 

In [ ]:
df_tb_stocks['stock_final'] = df_tb_stocks.groupby(['product_id'])['stock_final'].transform(scale_group)

In [ ]:
# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')

#Convertir 'periodo' a formato de fecha y Calcular el trimestre desde 'periodo'
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
df_final["trimestre"] = df_final.periodo.dt.quarter

#Establecer 'fecha' como índice y convertir a período mensual
#################df_final.set_index('fecha', inplace=True)
##################df_final.index = df_final.index.to_period('M')

In [ ]:
df_final = df_final[df_final['product_id'].isin(df_predecir['product_id'])] #Filtrar solo los productos a predecir

## FE

### Lags

In [ ]:
# Create lag variables for 'cust_request_qty', 'cust_request_tn', and 'tn'
n_lags = 36
for lag in range(1, n_lags + 1):
    #df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
    
    #df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
    
    #df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
   
    df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)

In [ ]:
#DELTA DE LAGS 2 ANTERIORES

for lag in range(1, n_lags-1):
    #df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
    #    df_final[f'cust_request_tn_lag_{lag+2}'] - df_final[f'cust_request_tn_lag_{lag}']
    #)
    #df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
    #    df_final[f'stock_final_lag_{lag+2}'] - df_final[f'stock_final_lag_{lag}']
    #)
    df_final[f'delta_tn_{lag}_{lag+2}'] = (
        df_final[f'tn_lag_{lag+2}'] - df_final[f'tn_lag_{lag}']
    )

In [ ]:
#DELTA DE LAGS 1 ANTERIOR

for lag in range(1, n_lags):
    #df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
    #    df_final[f'cust_request_tn_lag_{lag+1}'] - df_final[f'cust_request_tn_lag_{lag}']
    #)
    #df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
    #    df_final[f'stock_final_lag_{lag+1}'] - df_final[f'stock_final_lag_{lag}']
    #)
    df_final[f'delta_tn_{lag}_{lag+1}'] = (
        df_final[f'tn_lag_{lag+1}'] - df_final[f'tn_lag_{lag}']
    )

In [ ]:
#Mes actual / (lag 2 + lag 3) 

# Calcular el ratio del valor actual con respecto a la suma de los dos lags anteriores
for lag in range(3, n_lags + 1):
    """df_final[f'ratio_cust_request_tn_{lag}'] = (
        df_final[f'cust_request_tn_lag_{lag}'] / (
            df_final[f'cust_request_tn_lag_{lag-1}'] + df_final[f'cust_request_tn_lag_{lag-2}']
        )
    )
    df_final[f'ratio_stock_final_{lag}'] = (
        df_final[f'stock_final_lag_{lag}'] / (
            df_final[f'stock_final_lag_{lag-1}'] + df_final[f'stock_final_lag_{lag-2}']
        )
    )"""
    df_final[f'ratio_tn_{lag}'] = (
        df_final[f'tn_lag_{lag}'] / (
            df_final[f'tn_lag_{lag-1}'] + df_final[f'tn_lag_{lag-2}']
        )
    )

In [ ]:
#df_final.reset_index(inplace=True)

### Medias Móviles

In [ ]:
#MEDIAS MOVILES
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

### FE

In [ ]:
# Datetime features
df_final['year'] = df_final['periodo'].dt.year
df_final['month'] = df_final['periodo'].dt.month
df_final['quarter'] = df_final.periodo.dt.quarter

In [ ]:
#Variables Dummies si es el max o el min de cierta cantidad de meses
months = [3, 6, 9, 12]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for i in months:
    df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
    df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
    # Crear las dummies
    df_final[f'dummy_max_{i}m'] = np.where(df_final['tn'] == df_final[f'max_{i}m'], 1, 0)
    df_final[f'dummy_min_{i}m'] = np.where(df_final['tn'] == df_final[f'min_{i}m'], 1, 0)

In [ ]:
#Calcular ventas por trimestre
df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
#Calcular ventas por trimestre por cliente
df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')

In [ ]:
df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')

### FE variables externas

In [ ]:
# Leer el archivo exportado
df_exported = pd.read_excel(DATOS_DIR+'23variables_externas.xlsx')

# Asegúrate de que las columnas de fecha estén en el formato datetime
df_exported['fecha'] = pd.to_datetime(df_exported['fecha'])

In [ ]:
df_exported = pl.DataFrame(df_exported)

In [ ]:
import gc
gc.collect()

In [ ]:
#df_final = df_final.reset_index()

In [ ]:
df_final_copy = pl.DataFrame(df_final)

In [ ]:
df_final_copy = df_final_copy.join(df_exported, 
                 left_on=['fecha'],
                 right_on=['fecha'],
                 how='left'
                )

In [ ]:
df_final = df_final_copy

In [ ]:
#df_final['plan_precios_cuidados'] =df_final['plan_precios_cuidados'].astype(bool)
#df_final['dias_fin_trimestre'] = df_final['dias_fin_trimestre'].dt.days.astype(int) #TARDA!
df_final = df_final.to_pandas()
df_final = df_final.drop(columns=['fecha'])

In [ ]:
# #Cambiar las variables categoricas y hacer one-hot encoding

df_final["cat1"] = df_final["cat1"].astype("category")
df_final["cat2"] = df_final["cat2"].astype("category")
df_final["cat3"] = df_final["cat3"].astype("category")
df_final["brand"] = df_final["brand"].astype("category")
df_final["descripcion"] = df_final["descripcion"].astype("category")

# # Encode categorical variables explicitly. One-hot encoding
cat1_dummies = pd.get_dummies(df_final['cat1'], prefix='cat1', drop_first=True)
cat2_dummies = pd.get_dummies(df_final['cat2'], prefix='cat2', drop_first=True)
cat3_dummies = pd.get_dummies(df_final['cat3'], prefix='cat3', drop_first=True)
brand_dummies = pd.get_dummies(df_final['brand'], prefix='brand', drop_first=True)
#descripcion_dummies = pd.get_dummies(df_final['descripcion'], prefix='descripcion', drop_first=True)

# # Concatenate the dummy variables to the DataFrame and drop the original categorical columns
df_final= pd.concat([df_final, cat1_dummies, cat2_dummies, cat3_dummies, brand_dummies], axis=1)
df_final.drop(columns=['cat1', 'cat2', 'cat3', 'brand'], inplace=True)

df_final.set_index('periodo', inplace=True)
df_final.index = df_final.index.to_period('M')
df_final.sort_index(inplace=True)


In [ ]:
import pickle

# Assuming scalers_dict is the dictionary containing the scalers
# Convert the scalers_dict to binary file
#with open(DATOS_DIR+'/scalers.pkl', 'wb') as file:#
#     pickle.dump(scalers, file)

df_final.to_parquet(DATOS_DIR+'/FE_07_dataset.parquet', engine='pyarrow')  